# Azure OpenAI | Azure OpenAIこのノートブックでは、[Azure OpenAI](https://aka.ms/azure-openai)を使用してLangchainを使う方法について説明します。> This notebook goes over how to use Langchain with [Azure OpenAI](https://aka.ms/azure-openai).Azure OpenAI APIはOpenAIのAPIと互換性があります。`openai` Pythonパッケージを使用すると、OpenAIとAzure OpenAIの両方を簡単に使用できます。以下に記載された例外を除き、OpenAIを呼び出すのと同じ方法でAzure OpenAIを呼び出すことができます。> The Azure OpenAI API is compatible with OpenAI's API.  The `openai` Python package makes it easy to use both OpenAI and Azure OpenAI.  You can call Azure OpenAI the same way you call OpenAI with the exceptions noted below.## API configuration | API設定`openai` パッケージを環境変数を使用して Azure OpenAI で使用するように設定できます。以下は `bash` 用です：> You can configure the `openai` package to use Azure OpenAI using environment variables.  The following is for `bash`:```bash# Set this to `azure`export OPENAI_API_TYPE=azure# The API version you want to use: set this to `2023-05-15` for the released version.export OPENAI_API_VERSION=2023-05-15# The base URL for your Azure OpenAI resource.  You can find this in the Azure portal under your Azure OpenAI resource.export OPENAI_API_BASE=https://your-resource-name.openai.azure.com# The API key for your Azure OpenAI resource.  You can find this in the Azure portal under your Azure OpenAI resource.export OPENAI_API_KEY=<your Azure OpenAI API key>```あるいは、実行中のPython環境内で直接APIを設定することもできます：> Alternatively, you can configure the API right within your running Python environment:```pythonimport osos.environ["OPENAI_API_TYPE"] = "azure"```## Azure Active Directory Authentication | Azure Active Directory 認証Azure OpenAIに認証する方法は2つあります：> There are two ways you can authenticate to Azure OpenAI:* APIキー  > API Key* Azure Active Directory (AAD)  > Azure Active Directory (AAD)APIキーの使用は、スタートする上で最も簡単な方法です。Azureポータル内のあなたのAzure OpenAIリソースにあるAPIキーを見つけることができます。> Using the API key is the easiest way to get started. You can find your API key in the Azure portal under your Azure OpenAI resource.しかし、複雑なセキュリティ要件がある場合は、Azure Active Directory (AAD) の使用を検討してください。Azure OpenAIでAADを使用する方法についての詳細は[こちら](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/managed-identity)で確認できます。> However, if you have complex security requirements - you may want to use Azure Active Directory. You can find more information on how to use AAD with Azure OpenAI [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/managed-identity).ローカルで開発を行う場合、Azure CLIをインストールし、ログインしている必要があります。Azure CLIは[こちら](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli)からインストールできます。インストール後、`az login`コマンドを実行してログインしてください。> If you are developing locally, you will need to have the Azure CLI installed and be logged in. You can install the Azure CLI [here](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli). Then, run `az login` to log in.Azure OpenAIリソースに`Cognitive Services OpenAI User`というAzureロール割り当てを追加してください。これにより、Azure OpenAIを使用するためのAADからのトークンを取得できるようになります。このロール割り当ては、ユーザー、グループ、サービスプリンシパル、または管理されたアイデンティティに付与することができます。Azure OpenAIのRBACロールについての詳細は[こちら](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/role-based-access-control)をご覧ください。> Add a role an Azure role assignment `Cognitive Services OpenAI User` scoped to your Azure OpenAI resource. This will allow you to get a token from AAD to use with Azure OpenAI. You can grant this role assignment to a user, group, service principal, or managed identity. For more information about Azure OpenAI RBAC roles see [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/role-based-access-control).LangChainでPythonにおいてAADを利用するには、`azure-identity`パッケージをインストールしてください。次に、`OPENAI_API_TYPE`を`azure_ad`に設定します。その後、以下に示すように`DefaultAzureCredential`クラスを使用して`get_token`を呼び出し、AADからトークンを取得します。最後に、トークン値を環境変数`OPENAI_API_KEY`に設定してください。> To use AAD in Python with LangChain, install the `azure-identity` package. Then, set `OPENAI_API_TYPE` to `azure_ad`. Next, use the `DefaultAzureCredential` class to get a token from AAD by calling `get_token` as shown below. Finally, set the `OPENAI_API_KEY` environment variable to the token value.```pythonimport osfrom azure.identity import DefaultAzureCredential# Get the Azure Credentialcredential = DefaultAzureCredential()# Set the API type to `azure_ad`os.environ["OPENAI_API_TYPE"] = "azure_ad"# Set the API_KEY to the token from the Azure credentialos.environ["OPENAI_API_KEY"] = credential.get_token("https://cognitiveservices.azure.com/.default").token````DefaultAzureCredential` クラスは、AAD認証を始めるための簡単な方法です。必要に応じて、クレデンシャルチェーンをカスタマイズすることも可能です。以下に示す例では、最初に Managed Identity を試み、それが利用できない場合は Azure CLI にフォールバックします。これは、Azure でコードを実行しつつ、ローカルでの開発を行いたい場合に便利です。> The `DefaultAzureCredential` class is an easy way to get started with AAD authentication. You can also customize the credential chain if necessary. In the example shown below, we first try Managed Identity, then fall back to the Azure CLI. This is useful if you are running your code in Azure, but want to develop locally.```pythonfrom azure.identity import ChainedTokenCredential, ManagedIdentityCredential, AzureCliCredentialcredential = ChainedTokenCredential(    ManagedIdentityCredential(),    AzureCliCredential())```## Deployments | デプロイメントAzure OpenAIでは、一般的なGPT-3およびCodexモデルのデプロイメントを自分で設定します。APIを呼び出す際には、使用したいデプロイメントを指定する必要があります。> With Azure OpenAI, you set up your own deployments of the common GPT-3 and Codex models.  When calling the API, you need to specify the deployment you want to use.\***注意**: これらのドキュメントはAzureのテキスト補完モデル用です。GPT-4のようなモデルはチャットモデルであり、少し異なるインターフェースを持っています。これらは`AzureChatOpenAI`クラスを通じてアクセスできます。Azureチャットに関するドキュメントについては、[Azure Chat OpenAI documentation](/docs/integrations/chat/azure_chat_openai)をご覧ください。> ***Note**: These docs are for the Azure text completion models. Models like GPT-4 are chat models. They have a slightly different interface, and can be accessed via the `AzureChatOpenAI` class. For docs on Azure chat see [Azure Chat OpenAI documentation](/docs/integrations/chat/azure_chat_openai).*例えば、あなたのデプロイメント名が `text-davinci-002-prod` だとしましょう。`openai` Python APIでは、`engine` パラメータを使用してこのデプロイメントを指定できます。例えば：> Let's say your deployment name is `text-davinci-002-prod`.  In the `openai` Python API, you can specify this deployment with the `engine` parameter.  For example:```pythonimport openairesponse = openai.Completion.create(    engine="text-davinci-002-prod",    prompt="This is a test",    max_tokens=5)```

In [ ]:
!pip install openai

In [3]:
import os

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_BASE"] = "..."
os.environ["OPENAI_API_KEY"] = "..."

In [4]:
# Import Azure OpenAI
from langchain.llms import AzureOpenAI

In [5]:
# Create an instance of Azure OpenAI
# Replace the deployment name with your own
llm = AzureOpenAI(
    deployment_name="td2",
    model_name="gpt-3.5-turbo-instruct",
)

In [6]:
# Run the LLM
llm("Tell me a joke")

"\n\nWhy couldn't the bicycle stand up by itself? Because it was...two tired!"

LLMを出力して、そのカスタム出力を見ることもできます。> We can also print the LLM and see its custom print.

In [4]:
print(llm)

AzureOpenAI
Params: {'deployment_name': 'text-davinci-002', 'model_name': 'text-davinci-002', 'temperature': 0.7, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'best_of': 1}
